# 개요

- 텐서플로우에서 데이터 플로우 그래프 및 기타 부수적인 시각화 작업을 할수 있는 장치
- 텐서보드 
  - 코랩용
  - 아나콘다 베이스(일반 파이썬) 

# 텐서보드 모듈 가져오기

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [ ]:
#%pip list

# tensorboard                   1.15.0
# 코랩용 텐서보드 설치
!pip install tensorboardcolab

  Created wheel for tensorboardcolab: filename=tensorboardcolab-0.0.22-py3-none-any.whl size=3857 sha256=1237ea25714ed11c3897db340240d35121fb25cec6697ad733dece98702c8719
  Stored in directory: /root/.cache/pip/wheels/69/4e/4a/1c6c267395cb10edded1050df12af165d3254cfce324e80941
Successfully built tensorboardcolab


In [ ]:
# 텐서 보드 가져오기
from tensorboardcolab import *

Using TensorFlow backend.


In [ ]:
import os, shutil

# 텐서보드 설정

In [ ]:
# 텐서보드에 필요한 데이터를 저장하는 공간 지정
try:
  # 기존 자료 제거
  shutil.rmtree( './Graph', ignore_errors=True )
  # 생성
  os.mkdir('./Graph')
except Exception as e:
  print( e )

In [ ]:
# 코랩에 사용하는 텐서보드  생성
tbc = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://4899-35-202-100-97.ngrok.io


# 훈련 및 시각화

In [ ]:
# part1 : Data Flow Graph 생성
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

mnist     = input_data.read_data_sets( './data/mnist/', one_hot=True )
PIXEL     = mnist.train.images.shape[1]
PIXEL_H   = int( np.sqrt(PIXEL) )
PIXEL_W   = PIXEL_H
LABEL_NUM = mnist.train.labels.shape[-1]

x         = tf.placeholder( tf.float32, shape=(None, PIXEL), name='x' )

def make_FilterByWeight( name, shape ):  
  name = f'{name}_W'  
  tmp  = tf.truncated_normal(  shape, stddev=0.1 )  
  W    = tf.Variable( initial_value=tmp, name=name )
  return W;
def make_Bias( name, shape, value ):  
  name = f'{name}_b'  
  tmp  = tf.constant( value, shape=shape )  
  b    = tf.Variable( initial_value=tmp, name=name )
  return b
def make_conv2d( name, x, W ):  
  return tf.nn.conv2d(  x, filter=W, strides=[1,1,1,1], padding='SAME', name=name )

layer_name    = 'conv_1f'
with tf.name_scope( layer_name ) as scope:
  conv_1f_W   = make_FilterByWeight( layer_name, (5,5,1,32) )
  conv_1f_b   = make_Bias( layer_name, shape=(32,), value=0.1 )
  x_4d        = tf.reshape( x, ( -1, PIXEL_H, PIXEL_W, 1) )
  conv_1f     = make_conv2d( layer_name, x_4d,  conv_1f_W ) + conv_1f_b
  act_conv_1f = tf.nn.relu( conv_1f )

def make_MaxPooling( name, x ):
  name        = f'{name}_max'
  return tf.nn.max_pool(  x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name=name)  

layer_name    = 'pooling_1f'
with tf.name_scope( layer_name ) as scope:  
  pool_1f     = make_MaxPooling( layer_name, act_conv_1f )

layer_name     = 'conv_2f'
with tf.name_scope( layer_name ) as scope:  
  conv_2f_W    = make_FilterByWeight( layer_name, (5,5,32,32*2) )
  conv_2f_b    = make_Bias( layer_name, shape=(32*2,), value=0.1 )  
  conv_2f      = make_conv2d( layer_name, pool_1f,  conv_2f_W ) + conv_2f_b
  act_conv_2f  = tf.nn.relu( conv_2f )

layer_name     = 'pooling_2f'
with tf.name_scope( layer_name ) as scope:  
  pool_2f      = make_MaxPooling( layer_name, act_conv_2f )

layer_name = 'fc'
with tf.name_scope( layer_name ) as scope:  
  _, h, w, ch = pool_2f.shape
  in_channels = h * w * ch
  ch_size     = 1024
  tmp_x       = tf.reshape( pool_2f, ( -1,  in_channels) )
  fc_W        = make_FilterByWeight( layer_name, (in_channels, ch_size) )
  fc_b        = make_Bias( layer_name, (ch_size,), 0.1 )
  fc          = tf.matmul( tmp_x, fc_W ) + fc_b
  act_fc      = tf.nn.relu( fc )

layer_name        = 'act_fc_dropout'
with tf.name_scope( layer_name ) as scope:  
  keep_prob       = tf.placeholder( tf.float32 )
  act_fc_dropout  = tf.nn.dropout( act_fc, rate=1-keep_prob ) 

layer_name = 'output'
with tf.name_scope( layer_name ) as scope:  
  _, in_ch = act_fc_dropout.shape
  y_W      = make_FilterByWeight( layer_name, ( in_ch, LABEL_NUM )  )
  y_b      = make_Bias( layer_name,  (LABEL_NUM,), 0.1 )
  y_conv   = tf.matmul( act_fc_dropout, y_W ) + y_b
  y_conv   = tf.nn.softmax( y_conv )

y_ = tf.placeholder( tf.float32, shape=(None, LABEL_NUM), name='y_' )

layer_name      = 'loss'
with tf.name_scope( layer_name ) as scope:
  cross_entropy = -tf.reduce_sum( y_ * tf.log( y_conv )  )

layer_name = 'adam'
with tf.name_scope( layer_name ) as scope:  
  optimzer = tf.train.AdamOptimizer()  
  train    = optimzer.minimize( cross_entropy )

layer_name = 'predict'
with tf.name_scope( layer_name ) as scope:
  predict  = tf.equal( tf.argmax( y_conv, 1 ) , tf.argmax( y_, 1 ) )
  accuracy = tf.reduce_mean( tf.cast( predict, tf.float32 ) )

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [ ]:
# part2 : 실제 학습/평가 수행
def make_Feed_Dict( x_data, labels, prob ):    
  return {
      x:x_data,
      y_:labels,
      keep_prob:prob
  }

with tf.device('/device:GPU:0'):
  with tf.Session() as sess:
    TRAIN_COUNT  = 3000
    BATCH_SIZE   = 50    
    VERBOSE_TERM = 100
    sess.run( tf.global_variables_initializer() )  
    test_fd = make_Feed_Dict( mnist.test.images,  mnist.test.labels, 1.0 )
    for step in range( TRAIN_COUNT ):
      batch = mnist.train.next_batch( BATCH_SIZE )        
      train_fd = make_Feed_Dict( batch[0], batch[1], 0.1 )
      acc, _, loss_log = sess.run( [accuracy, train, cross_entropy] , feed_dict=train_fd )
      #loss_logs.append( loss_log )
      if step % VERBOSE_TERM == 0:      
        acc = sess.run( accuracy, feed_dict=test_fd )
        print( f'step={step:4} acc={acc:20}  loss={loss_log:20}' )          
      
    acc = sess.run( accuracy, feed_dict=test_fd )
    print( f'step={step:4} acc={acc:20}  loss={loss_log:20}' )
    
    # 텐서보드 기록 작업
    writer = tbc.get_writer()
    writer.add_graph( sess.graph )
    writer.flush()

# tbc 닫기
tbc.close()

step=   0 acc= 0.09799999743700027  loss=                 nan
step= 100 acc= 0.09799999743700027  loss=                 nan
step= 200 acc= 0.09799999743700027  loss=                 nan
step= 300 acc= 0.09799999743700027  loss=                 nan
step= 400 acc= 0.09799999743700027  loss=                 nan
step= 500 acc= 0.09799999743700027  loss=                 nan
step= 600 acc= 0.09799999743700027  loss=                 nan
step= 700 acc= 0.09799999743700027  loss=                 nan
step= 800 acc= 0.09799999743700027  loss=                 nan
step= 900 acc= 0.09799999743700027  loss=                 nan
step=1000 acc= 0.09799999743700027  loss=                 nan
step=1100 acc= 0.09799999743700027  loss=                 nan
step=1200 acc= 0.09799999743700027  loss=                 nan
step=1300 acc= 0.09799999743700027  loss=                 nan
step=1400 acc= 0.09799999743700027  loss=                 nan
step=1500 acc= 0.09799999743700027  loss=                 nan
step=160